---
# Gmsh Tutorials
- this notebook aims to develop my skills using Gmsh Python API
- python-specific reference: https://gmsh.info/doc/texinfo/gmsh.html#x1


In [28]:
# Tutorial 1 - Geometry basics, elementary entities, physical groups
# https://gitlab.onelab.info/gmsh/gmsh/blob/gmsh_4_11_1/tutorials/python/t1.py
import gmsh
import sys

# Before using any functions in the Python API, Gmsh must be initialized:
if not gmsh.is_initialized():
    gmsh.initialize()

# Next we add a new model named "t1":
Name = "test_mesh"
gmsh.model.add(Name)


# 1.POINTS: The first type of `elementary entity' in Gmsh is a `Point'. To create a point
# the Python API function is gmsh.model.geo.addPoint()  or gmsh.model.geo.add_point()
# 3 arguments are the point coordinates (x, y, z)
# - the next (optional) argument is the target mesh size close to the point
# - the last (optional) argument is the point tag/identifier
lc = 1  # target mesh size
z = 0   # fixed z coordinate
gmsh.model.geo.addPoint(0, 0, z, lc, 1)
gmsh.model.geo.addPoint(0, 10, 0, lc, 2)
gmsh.model.geo.addPoint(10, 10, 0, lc, 3)
gmsh.model.geo.addPoint(10, 0, 0, lc, 4)

# 2.LINES: Curves are second type of elementery entities
# Straight line segments: the first 2 arguments are
# - point tags (the start and end points of the line)
# - the last (optional) is the line tag.
gmsh.model.geo.addLine(1, 2, 1)
gmsh.model.geo.addLine(2, 3, 2)
gmsh.model.geo.addLine(3, 4, 3)
gmsh.model.geo.addLine(4, 1, 4)


# 3.SURFACES: to define a simple rectangular surface, curve loop (ordered list of connected curves)
# has to be defined.  Arguments are
# - list of curve tags
# - the curve loop tag (which must be unique
# amongst curve loops) as the second (optional) argument:
gmsh.model.geo.addCurveLoop([1, 2, 3, 4], 1)

# add the surface as a list of curve loops: outline + holes(optional)
gmsh.model.geo.addPlaneSurface([1], 1)


# Before it can be meshed (and, more generally, before it can be used by API
# functions outside of the built-in CAD kernel functions), the CAD entities must
# be synchronized with the Gmsh model
gmsh.model.geo.synchronize()


# physical groups: mathematical ("domain", "boundary"), functional ("left wing", "fuselage") ...
# Here we define a physical curve that groups the left, right and bottom curves
# in a single group (with prescribed tag 5); and a physical surface with name
# "My surface" (with an automatic tag) containing the geometrical surface 1:
gmsh.model.addPhysicalGroup(1, [1, 3, 4], 5)              # curves physical group
gmsh.model.addPhysicalGroup(2, [1], name="My surface")  # surfaces physical group


# generate a 2D mesh
gmsh.model.mesh.generate(2)


# save it to disk (by default only physical groups are saved)
gmsh.write(f"./meshes/{Name}.msh")
gmsh.write(f"./meshes/{Name}.stl")

# By default, if physical groups are defined, Gmsh will export in
# the output mesh file only those elements that belong to at least one physical
# group. To force Gmsh to save all elements, you can use:
#    gmsh.option.setNumber("Mesh.SaveAll", 1)


# visualize the model in the GUI
gmsh.fltk.run()

# when finished with Gmsh Python API:
gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 50%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.000488143s, CPU 0.000736s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.00486965s, CPU 0.004591s)
Info    : 143 nodes 288 elements
Info    : Writing './meshes/test_mesh.msh'...
Info    : Done writing './meshes/test_mesh.msh'
Info    : Writing './meshes/test_mesh.stl'...
Info    : Done writing './meshes/test_mesh.stl'
-------------------------------------------------------
Version       : 4.11.1
License       : GNU General Public License
Build OS      : MacOSX-sdk
Build date    : 20230510
Build host    : Mac-1683750338053.local
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack MathEx[contrib] Mesh Metis[cont

In [6]:
# Tutorial 2 - Transformations, extruded geometries, volumes

import gmsh
import sys
import numpy as np


# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()

# add new model
Name = "test_mesh2"
gmsh.model.add(Name)

# setting up the geometry
lc = 1e-2  # target mesh size
gmsh.model.geo.addPoint(0, 0, 0, lc, 1)
gmsh.model.geo.addPoint(.1, 0, 0, lc, 2)
gmsh.model.geo.addPoint(.1, .3, 0, lc, 3)
gmsh.model.geo.addPoint(0, .3, 0, lc, 4)
gmsh.model.geo.addLine(1, 2, 1)
gmsh.model.geo.addLine(3, 2, 2)
gmsh.model.geo.addLine(3, 4, 3)
gmsh.model.geo.addLine(4, 1, 4)
gmsh.model.geo.addCurveLoop([4, 1, -2, 3], 1)
gmsh.model.geo.addPlaneSurface([1], 1)
gmsh.model.geo.synchronize()
gmsh.model.addPhysicalGroup(1, [1, 2, 4], 5)  # left, right and bottom
gmsh.model.addPhysicalGroup(2, [1], name="My surface")  # surface group


#add new points and curves
gmsh.model.geo.addPoint(0, .4, 0, lc, 5)
gmsh.model.geo.addLine(4, 5, 5)


# transformation: translation
gmsh.model.geo.translate([(0, 5)], -0.02, 0, 0)  # translate([(dimension, tag)], dx, dy, dz)

# transformation: rotation
gmsh.model.geo.rotate([(0, 5)], 0, 0.3, 0, 0, 0, 1, -np.pi / 4)  # around (0, 0.3, 0) along the z axis by -Pi/4

# transformation: duplication
ov = gmsh.model.geo.copy([(0, 3)])        # copy([(dim, tag)]) -> returns (dim, tag)
gmsh.model.geo.translate(ov, 0, 0.05, 0)  # translation of the duplicated point


# creating lines to the duplicated point
gmsh.model.geo.addLine(3, ov[0][1], 7)
gmsh.model.geo.addLine(ov[0][1], 5, 8)
gmsh.model.geo.addCurveLoop([5, -8, -7, 3], 10)
gmsh.model.geo.addPlaneSurface([10], 11)

# duplicating and translating surfaces (surface 1 and surface 11)
ov = gmsh.model.geo.copy([(2, 1), (2, 11)])
gmsh.model.geo.translate(ov, 0.12, 0, 0)

print("New surfaces " + str(ov[0][1]) + " and " + str(ov[1][1]))  # printing the new tags

# Creating volumes 1: Manually
gmsh.model.geo.addPoint(0., 0.3, 0.12, lc, 100)
gmsh.model.geo.addPoint(0.1, 0.3, 0.12, lc, 101)
gmsh.model.geo.addPoint(0.1, 0.35, 0.12, lc, 102)

# accessing coordinates of points
gmsh.model.geo.synchronize()         # first synchronize the model
xyz = gmsh.model.getValue(0, 5, [])  # get coordinates of the point 5
gmsh.model.geo.addPoint(xyz[0], xyz[1], 0.12, lc, 103)  # add point above 5

gmsh.model.geo.addLine(4, 100, 110)
gmsh.model.geo.addLine(3, 101, 111)
gmsh.model.geo.addLine(6, 102, 112)
gmsh.model.geo.addLine(5, 103, 113)
gmsh.model.geo.addLine(103, 100, 114)
gmsh.model.geo.addLine(100, 101, 115)
gmsh.model.geo.addLine(101, 102, 116)
gmsh.model.geo.addLine(102, 103, 117)

gmsh.model.geo.addCurveLoop([115, -111, 3, 110], 118)
gmsh.model.geo.addPlaneSurface([118], 119)
gmsh.model.geo.addCurveLoop([111, 116, -112, -7], 120)
gmsh.model.geo.addPlaneSurface([120], 121)
gmsh.model.geo.addCurveLoop([112, 117, -113, -8], 122)
gmsh.model.geo.addPlaneSurface([122], 123)
gmsh.model.geo.addCurveLoop([114, -110, 5, 113], 124)
gmsh.model.geo.addPlaneSurface([124], 125)
gmsh.model.geo.addCurveLoop([115, 116, 117, 114], 126)
gmsh.model.geo.addPlaneSurface([126], 127)

gmsh.model.geo.addSurfaceLoop([127, 119, 121, 123, 125, 11], 128)
gmsh.model.geo.addVolume([128], 129)



# Creating volumes 2: Extruding surface
ov2 = gmsh.model.geo.extrude([ov[1]], 0, 0, 0.12)  # extrude([(dim, tag), vector])


# setting up multiple mesh sizes with setSize()
gmsh.model.geo.mesh.setSize([(0, 103), (0, 105), (0, 109), (0, 102), (0, 28),
                             (0, 24), (0, 6), (0, 5)], lc * 3)


# synchronize with the model
gmsh.model.geo.synchronize()


# creating a physical group
gmsh.model.addPhysicalGroup(3, [129, 130], 1, "The volume")


# generate mesh:
gmsh.model.mesh.generate(3)

# save the mesh:
gmsh.write(f"./meshes/{Name}.msh")
gmsh.write(f"./meshes/{Name}.stl")
# gmsh.write(f"{Name}.geo_unrolled")  # `Gmsh Unrolled GEO' format


# visualize the model in the GUI
gmsh.fltk.run()

# when finished
gmsh.finalize()


New surfaces 12 and 17
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 20%] Meshing curve 5 (Line)
Info    : [ 20%] Meshing curve 7 (Line)
Info    : [ 20%] Meshing curve 8 (Line)
Info    : [ 30%] Meshing curve 13 (Line)
Info    : [ 30%] Meshing curve 14 (Line)
Info    : [ 30%] Meshing curve 15 (Line)
Info    : [ 40%] Meshing curve 16 (Line)
Info    : [ 40%] Meshing curve 18 (Line)
Info    : [ 40%] Meshing curve 19 (Line)
Info    : [ 50%] Meshing curve 20 (Line)
Info    : [ 50%] Meshing curve 110 (Line)
Info    : [ 50%] Meshing curve 111 (Line)
Info    : [ 60%] Meshing curve 112 (Line)
Info    : [ 60%] Meshing curve 113 (Line)
Info    : [ 60%] Meshing curve 114 (Line)
Info    : [ 70%] Meshing curve 115 (Line)
Info    : [ 70%] Meshing curve 116 (Line)
Info    : [ 70%] Meshing curve 117 (Line)
Info    : [ 80%] Meshing curve 131 (Line)
Info    